## Dataset
- You will be using a modified version of the FairFace dataset (https://github.com/joojs/fairface). This is a set of 86,744 training face images and 10,954 validation face images. 
- In order to decrease the training time I converted all images to gray scale and resized them to 32 × 32. Each face has 3 different attributes which can be used for a classification task: race, gender, and age. All files can be found in the zip file on Canvas. The train folder contains the training images and the fairface label train.csv file contains all the label. There is a similar folder and file for the validation set.
- As the three different attributes have a different number of possible values, your final layers for each classifier will vary. For each of the networks below please attempt to classify 2 of the attributes (you can choose which).

## Imports

In [1]:
#Tensor imports
import tensorflow as tf
from tensorflow import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.callbacks import TensorBoard

#Pillow Imports
from PIL import Image

#Import Pandas
import pandas as pd

#Import Numpy
import numpy as np

#Sci_Kit Imports
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix

#Import datetime
import datetime

## Load TensorBoard and Create Logs

In [4]:
%load_ext tensorboard
%reload_ext tensorboard
#log_folder = "logs"
log_folder = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## Task 1: Fully Connected Neural Network
1. Build a feed forward neural network with the following specifications (Test on two different tasks):
    - Hidden layer 1: 1024 neurons with hyperbolic tangent activation function in each neuron.
    - Hidden layer 2: 512 neurons, with sigmoid activation function in each of the neuron.
    - 100 neurons, with rectified linear activation function in each of the neuron.
    - Output layer: n (depending on the task) neurons representing the n classes, using the softmax activation function.
2. Using Min-Max scaling to scale the training dataset and using the same Min and Max values from the training set scale the test dataset (X−Xmin/Xmax−Xmin).
3. Using mini-batch gradient descent to optimize the loss function: “categorical cross-entropy” on the training dataset. Please record the loss value for each of the epochs and create an epoch-loss plot and an accuracy-loss plot for both the training and validation set.
4. Report the following:
    - Final classification accuracy.
    - The n-class confusion matrix.

In [5]:
def createFeedFoward(inputShape, outputSize, lr):
    model = Sequential()
    model.add(layers.Dense(1024, input_shape=inputShape, activation='tanh'))
    model.add(layers.Dense(512, activation='sigmoid'))
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(outputSize, activation='softmax'))
    opt = optimizers.SGD(learning_rate=lr)
    model.compile(loss='CategoricalCrossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [6]:
def createXY(imgPath, labelFile, num):
    #Create and normalize X
    X = []
    for i in range(num):
        fileName = imgPath + str(i+1) + '.jpg'
        img = Image.open(fileName)
        X.append(list(img.getdata()))
    scaler = MinMaxScaler()
    scaler.fit(X)
    X = scaler.transform(X)
    
    #Get labels
    label_df = pd.read_csv(labelFile)
    
    #Find unique labels and output size
    age_labels = label_df['age'].unique()
    race_labels = label_df['race'].unique()
    
    #Create Binary y arrays
    lb_age = LabelBinarizer(sparse_output=False)
    lb_race = LabelBinarizer(sparse_output=False)
    lb_age.fit(age_labels)
    lb_race.fit(race_labels)
    y_age = list(label_df['age'].head(num))
    y_race = list(label_df['race'].head(num))
    y_age = lb_age.transform(y_age)
    y_race = lb_age.transform(y_race)
    
    
    return X, y_age, y_race, age_labels, race_labels 

In [7]:
def printResults(predictions, labels, trueLabels):
    print(len(predictions))
    print(len(predictions[1]))
    print(len(labels))
    print(len(trueLabels))
    for i in range(len(predictions)):
        print("Label: " + trueLabels[i])
        for j in range(len(labels)):
            print("{:12}: {:10.2f}%".format(labels[j], (predictions[i][j] * 100)))

In [8]:
def getMax(values):
    maxes = [np.argmax(val) for val in values ]
    return maxes

In [9]:
callbacks = [TensorBoard(log_dir=log_folder,
                         histogram_freq=1,
                         write_graph=True,
                         write_images=True,
                         update_freq='epoch',
                         profile_batch=2,
                         embeddings_freq=1)]

In [10]:
X_train, y_age_train, y_race_train, age_labels, race_labels = createXY('project3_COSC525/train/', 'project3_COSC525/fairface_label_train.csv', 86744)
X_test, y_age_test, y_race_test, _ , _ = createXY('project3_COSC525/val/', 'project3_COSC525/fairface_label_val.csv', 10954)

In [11]:
def taskOne(X_train, y_train, X_test, y_test, lr, numEpochs, batchSize, port):
    model = createFeedFoward((1024,), len(y_train[0]), lr)
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=numEpochs, batch_size=batchSize, callbacks=callbacks)
    y_true = getMax(y_test)
    y_pred = getMax(model.predict(X_test))
    eval = tf.keras.metrics.Accuracy()
    eval.update_state(y_true, y_pred)
    print('Accuracy: ', eval.result().numpy())
    c_matrix = confusion_matrix(y_true, y_pred)
    print(c_matrix)
    %tensorboard --logdir logs --port=port

In [ ]:
taskOne(X_train, y_age_train, X_test, y_age_test, 0.001, 120, 100, 6009)

Epoch 1/120
868/868 [==============================] - 14s 14ms/step - loss: 1.8844 - accuracy: 0.2911 - val_loss: 1.8621 - val_accuracy: 0.3013
Epoch 2/120
868/868 [==============================] - 11s 13ms/step - loss: 1.8650 - accuracy: 0.2951 - val_loss: 1.8590 - val_accuracy: 0.3018
Epoch 3/120
868/868 [==============================] - 10s 12ms/step - loss: 1.8612 - accuracy: 0.2952 - val_loss: 1.8550 - val_accuracy: 0.3013
Epoch 4/120
868/868 [==============================] - 11s 13ms/step - loss: 1.8582 - accuracy: 0.2954 - val_loss: 1.8521 - val_accuracy: 0.3019
Epoch 5/120
868/868 [==============================] - 11s 13ms/step - loss: 1.8554 - accuracy: 0.2953 - val_loss: 1.8491 - val_accuracy: 0.3015
Epoch 6/120
868/868 [==============================] - 11s 13ms/step - loss: 1.8528 - accuracy: 0.2954 - val_loss: 1.8470 - val_accuracy: 0.3048
Epoch 7/120
868/868 [==============================] - 10s 12ms/step - loss: 1.8503 - accuracy: 0.2958 - val_loss: 1.8446 - val_ac

In [ ]:
taskOne(X_train, y_race_train, X_test, y_race_test, 0.001, 120, 100, 6009)

## Task 2: Small Convolutional Neural Network
- Build a convolutional neural network with the following specifications (Test on two different tasks):
    - Convolution layer having 40 feature detectors, with kernel size 5 x 5, and ReLU as the activation function, with stride 1 and no-padding.
    - A max-pooling layer with pool size 2x2.
    - Fully connected layer with 100 neurons, and ReLU as the activation function.
    - Output layer: n (depending on the task) neurons representing the n classes, using the softmax activation function. function for each of the 10 neurons.
2. Using Min-Max scaling to scale the training dataset and using the same Min and Max values from the training set scale the test dataset ( X−Xmin/Xmax−Xmin ).
3. Using mini-batch gradient descent to optimize the loss function: “categorical cross-entropy” on the training dataset. Please record the loss value for each of the epochs and create an epoch-loss plot and an accuracy-loss plot for both the training and validation set.
4. Report the following:
    - Final classification accuracy.
    - The n-class confusion matrix.

## Task 3: Your own Convolutional Neural Network
1. Build another convolutional neural network, where you choose all the parameters to see if you can get a higher accuracy.
2. Using Min-Max scaling to scale the training dataset and using the same Min and Max values from the training set scale the test dataset ( X−Xmin/Xmax−Xmin ).
3. Using mini-batch gradient descent to optimize the loss function: “categorical cross-entropy” on the training dataset. Please record the loss value for each of the epochs and create an epoch-loss plot and an accuracy-loss plot for both the training and validation set.
4. Report the following:
    - Final classification accuracy.
    - The n-class confusion matrix

## Task 4: Your own Convolutional Neural Network on both Tasks Simultaneously
1. Build another convolutional neural network, where you try and classify both tasks with a single network. After your flatten layer have two more fully connected layers for each “branch”. Note that in order to do so you will not be able to use the Sequential model.
2. Using Min-Max scaling to scale the training dataset and using the same Min and Max values from the training set scale the test dataset ( X−Xmin/Xmax−Xmin ).
3. Using mini-batch gradient descent to optimize the loss function: “categorical cross-entropy” on the training dataset. Please record the loss value for each of the epochs and create an epoch-loss plot and an accuracy-loss plot for both the training and validation set.
4. Report the following:
    - Final classification accuracy.
    - The n-class confusion matrix

## Task 5: Variational Auto Encoder (COSC 525 only)
1. Build a variational autoencoder with the following specifications (in this one you have a little more flexibility):
    - Should have at least two convolution layers in the encoder and 2 deconvolution layers in the decoder.
    - Latent dimension should be at least 5.
    - Loss should be either MSE or binary cross entropy.
2. Using Min-Max scaling to scale the training dataset and using the same Min and Max values from the training set scale the test dataset ( X−Xmin/Xmax−Xmin ).
3. Using mini-batch gradient descent to optimize the loss function on the training dataset. Please record the loss value for each of the epochs and create an epoch-loss plot and an accuracy-loss plot for both the training and validation set.
4. Qualitatively evaluate your model by generating a set of faces by randomly choosing 10 latent vectors and presenting the resulting images